In [1]:
import pandas as pd

In [2]:
from numpy import *


class MatchRating2:
    original_rating_name = "Original"
    avg_player_name = "AVG"
    max_iterations = 1000

    def __init__(self, data_sheet_file, rating_scale_base, rating_scale_divisor, rating_base, tie_in_weight):

        self.rating_scale_base = rating_scale_base
        self.rating_scale_divisor = rating_scale_divisor
        self.ranking_boost = rating_base
        self.tie_in_weight = tie_in_weight
        
        self.players: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Players", index_col="ShortName")
        self.matches: pd.DataFrame = pd.read_excel(data_sheet_file, sheet_name="Matches")
        self.matches["IsTieInMatch"] = False
        
        match_columns = self.matches.columns.to_list()
        self.winner_columns = [column for column in match_columns if column.startswith("Winner")]
        self.loser_columns = [column for column in match_columns if column.startswith("Loser")]

        print('Number of matches loaded:' + str(len(self.matches)))

        # --- Add tie-in matches ---
        cols = self.matches.columns
        first_winner = cols[cols.str.startswith("Winner")].values[0]
        first_loser = cols[cols.str.startswith("Loser")].values[0]
        
        tie_in_rows = []
        for player in self.players.index:
            tie_in_rows.append({first_winner: player, first_loser: MatchRating2.avg_player_name, "K": self.tie_in_weight})
            tie_in_rows.append({first_winner: MatchRating2.avg_player_name, first_loser: player, "K": self.tie_in_weight})

        tie_in_matches = pd.DataFrame(tie_in_rows)
        tie_in_matches["IsTieInMatch"] = True
        self.matches = pd.concat([self.matches, tie_in_matches], ignore_index=True)

        self.__player_ratings: pd.DataFrame = pd.DataFrame(columns=["Rating"], index=self.players.index)
        self.__player_ratings["Rating"] = 0.0
        

        self.winner_df = self.matches.reset_index().melt(id_vars=['index'], value_vars=self.winner_columns)
        self.winner_df.columns = ['Match', 'Winner', 'Player']
        self.winner_df = self.winner_df[self.winner_df["Player"].notna()]
        self.winner_df = self.winner_df.drop("Winner", axis=1)
        self.winner_df = self.winner_df.set_index("Player")

        self.loser_df = self.matches.reset_index().melt(id_vars=['index'], value_vars=self.loser_columns)
        self.loser_df.columns = ['Match', 'Loser', 'Player']
        self.loser_df = self.loser_df[self.loser_df["Player"].notna()]
        self.loser_df = self.loser_df.drop("Loser", axis=1)
        self.loser_df = self.loser_df.set_index("Player")
        
        

    def calculate_ratings(self, step_sizes: list, min_player_match_count: int):

        def apply_ranking_boost(rankings):
            rankings["Rating"] = rankings["Rating"] - rankings["Rating"].loc[MatchRating2.avg_player_name] + self.ranking_boost
            return rankings
        
        def remove_excluded_players(rankings):
            excluded_players = self.players[self.players["Excluded"] == True].index
            rankings = rankings.drop(excluded_players, errors="ignore")
            return rankings.drop(columns=["Excluded"])
        
        def add_player_metadata(rankings):
            return rankings.merge(self.players, left_index=True, right_index=True)
        
        def add_match_metadata(rankings):
            
            def get_wins_and_losses(player):
                matches = self.matches[self.matches["IsTieInMatch"] == False]
                wins = matches[self.winner_columns].apply(lambda x: (player in x.values), axis=1).sum()
                losses = matches[self.loser_columns].apply(lambda x: (player in x.values), axis=1).sum()
                return pd.Series([wins, losses], index=["Wins", "Losses"])
            
            rankings[["Wins", "Losses"]] = rankings["ShortName"].apply(get_wins_and_losses)
            return rankings
        
        def filter_min_match_count(rankings):
            return rankings[rankings["Wins"] + rankings["Losses"] >= min_player_match_count]

        def add_opponent_rating(rankings):

            def get_opponent_rating(player):
                matches = self.matches[(self.matches["IsTieInMatch"] == False) & self.matches.apply(lambda x: (player in x.values), axis=1)]
                matches = matches[self.winner_columns + self.loser_columns]

                def get_match_opponent_rating(match):
                    if player in match[self.winner_columns].values:
                        team_columns = self.winner_columns
                        opponent_columns = self.loser_columns
                    else:
                        team_columns = self.loser_columns
                        opponent_columns = self.winner_columns

                    team_players = match[team_columns].dropna().values
                    opponent_players = match[opponent_columns].dropna().values

                    player_team_ranking = rankings.loc[team_players, ["Rating"]].mean().values[0]
                    opponent_team_ranking = rankings.loc[opponent_players, ["Rating"]].mean().values[0]

                    return mean(opponent_team_ranking - player_team_ranking)

                opponent_ratings = matches.apply(get_match_opponent_rating, axis=1)
                return round(opponent_ratings.mean(), 0)
        
            rankings["AVG_Opponent_Rating"] = rankings["ShortName"].apply(get_opponent_rating)
            return rankings
        
        def finalize_rankings(rankings):
            rankings = rankings.sort_values(by="Rating", ascending=False).reset_index(drop=True)
            rankings.index += 1
            rankings["Rating"] = rankings["Rating"].round(0).astype(int)
            return rankings.reset_index()
        

        for step_size in step_sizes:
            rankings = self.__calculate_ratings(step_size)

        rankings["ShortName"] = rankings.index

        
        rankings = apply_ranking_boost(rankings)
        rankings = add_player_metadata(rankings)
        rankings = add_match_metadata(rankings)
        rankings = add_opponent_rating(rankings)
        rankings = remove_excluded_players(rankings)
        rankings = filter_min_match_count(rankings)
        rankings = finalize_rankings(rankings)

        return rankings


        
    
    def __calculate_ratings(self, step_size):
        
        total_match_weights  = self.matches["K"].sum()

        for i in range(MatchRating2.max_iterations):
            rating_table = self.__create_rating_table(step_size)
            
            match_score_accuracy = self.__calculate_match_score_accuracy(rating_table)        
            
            player_rating_accuracy = pow(match_score_accuracy.prod(), 1 / total_match_weights)
            accuracy_product = player_rating_accuracy.sort_values(ascending=False)

            best_accuracy = accuracy_product.index[0]
            print(f"Step Size={step_size}, Iteration {i + 1}, Accuracy: {round(accuracy_product.iloc[0], 5)}")

            if best_accuracy == MatchRating2.original_rating_name:
                return self.__player_ratings
            
            player, rating_change = best_accuracy            
            self.__player_ratings.loc[player, ["Rating"]] += rating_change
            
        return self.__player_ratings
    

    def __calculate_match_score_accuracy(self, rating_table: pd.DataFrame):

        winner_rating = self.winner_df.join(rating_table)
        winner_rating = winner_rating.groupby(by="Match").sum()

        loser_rating = self.loser_df.join(rating_table)
        loser_rating = loser_rating.groupby(by="Match").sum()
        
        win_rate = winner_rating / (winner_rating + loser_rating)

        rating_accuracy = win_rate.pow(self.matches["K"], axis=0)

        return rating_accuracy
    

    def __create_rating_table(self, step_size):
        
        ratings = []
        columns = []

        original_rating = self.__player_ratings["Rating"]        

        def add_rating(rating, name):
            tmp = rating.apply(lambda x: pow(self.rating_scale_base, (x / self.rating_scale_divisor)))
            ratings.append(tmp)
            columns.append(name)

        def add_alternative_rating(player, rating_change):
            alternative_rating = original_rating.copy()
            alternative_rating.loc[player] += rating_change
                
            add_rating(alternative_rating, (player, rating_change))

        add_rating(original_rating, MatchRating2.original_rating_name)

        for player in self.players.index:
            add_alternative_rating(player, step_size)
            add_alternative_rating(player, -step_size)
            
        rating_table = pd.concat(ratings, axis=1)
        rating_table.columns = columns

        return rating_table

In [3]:
rating_system = MatchRating2(
    # data_sheet_file="Data/Test.xlsx",
    data_sheet_file="Data/UCH_Data2.xlsx",
    # data_sheet_file="Data/BPLData.xlsx",
    rating_scale_base=10,
    rating_scale_divisor=400,
    rating_base=1000,
    tie_in_weight=0.5
)

Number of matches loaded:372


In [4]:
rankings = rating_system.calculate_ratings(
    step_sizes=[256, 128, 64, 32, 16, 8, 4, 2, 1, 0.5, 0.1],
    min_player_match_count=0
)
rankings

Step Size=256, Iteration 1, Accuracy: 0.51439
Step Size=256, Iteration 2, Accuracy: 0.52644
Step Size=256, Iteration 3, Accuracy: 0.54138
Step Size=256, Iteration 4, Accuracy: 0.56524
Step Size=256, Iteration 5, Accuracy: 0.5844
Step Size=256, Iteration 6, Accuracy: 0.58979
Step Size=256, Iteration 7, Accuracy: 0.59515
Step Size=256, Iteration 8, Accuracy: 0.60056
Step Size=256, Iteration 9, Accuracy: 0.60593
Step Size=256, Iteration 10, Accuracy: 0.61152
Step Size=256, Iteration 11, Accuracy: 0.6163
Step Size=256, Iteration 12, Accuracy: 0.61986
Step Size=256, Iteration 13, Accuracy: 0.62343
Step Size=256, Iteration 14, Accuracy: 0.62703
Step Size=256, Iteration 15, Accuracy: 0.62968
Step Size=256, Iteration 16, Accuracy: 0.63234
Step Size=256, Iteration 17, Accuracy: 0.63546
Step Size=256, Iteration 18, Accuracy: 0.63814
Step Size=256, Iteration 19, Accuracy: 0.64031
Step Size=256, Iteration 20, Accuracy: 0.64217
Step Size=256, Iteration 21, Accuracy: 0.64381
Step Size=256, Iteration

,index,Rating,ShortName,Name,Wins,Losses,AVG_Opponent_Rating
0,1,1777,JP,ja_paucia,44,12,-353.0
1,2,1720,MRF,Morfin,60,24,-250.0
2,3,1716,IEY,Gigi,32,8,-348.0
3,4,1660,RAI,Raito,52,24,-227.0
4,5,1544,TNR,TN'ramon,20,16,-142.0
5,6,1427,MAD,MAD MAN,24,20,-74.0
6,7,1406,EMB,Ember,8,8,-11.0
7,8,1339,VB,Variety_Bear,20,16,-172.0
8,9,1326,SZP,szop,24,20,-53.0
9,10,1176,VOI,Void,4,8,207.0
